In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

import pandas as pd

import locale
from datetime import datetime, timedelta
import time

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://www.sh-tourismus.de/veranstaltungskalender')

In [4]:
# Wait until the button is present
wait = WebDriverWait(driver, 10)

In [5]:
# Wait for the iframe to be present and switch to it
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe[src*="tashwhl.infomaxnet.de"]'))
)
driver.switch_to.frame(iframe)

In [6]:
#while True:
for i in range(400): #TODO konfigurierbar, ca 300 events pro tag bisschen puffer, 10 events pro mal clicken laden also 30 mal klicken pro tag a 10 tage incl puffer müsste so ok sein, paar spätere mit rein
    try:
        button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, '-IMXEVENT-lazyLoadButton')))
        # Click the button
        button.click()
    except StaleElementReferenceException:
        # Handle the case where the element is no longer attached to the DOM
        continue
    except TimeoutException:
        # Handle the case where the element is not found within the timeout
        print("Button not found or not clickable within the timeout period.")
        break

KeyboardInterrupt: 

In [10]:
# Find all article elements representing events
articles = driver.find_elements(By.CSS_SELECTOR, 'article.-IMXEVNT-listElement')

# Iterate through each article and extract information
for article in articles:
    # Extract the required information
    event_category = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__subline').text
    event_title = article.find_element(By.CSS_SELECTOR, 'span.-IMXEVNT-title').get_attribute('data-uppertitle')
    event_date_location = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__info').text
    event_source = article.find_element(By.CSS_SELECTOR, 'h2 a').get_attribute('href')

    # Print the extracted information
    print(f"Category: {event_category}")
    print(f"Title: {event_title}")
    print(f"Date and Location: {event_date_location}")
    print(f"Source: {event_source}")
    #print(f"Description: {event_description}")
    print('-' * 40)

Category: Sonstiges
Title: DAY SPA MIT MEERBLICK
Date and Location: 26.11.2024 / 07:00 - 22:00 Uhr / ATLANTIC Grand Hotel Travemünde, Travemünde
Source: https://tashwhl.infomaxnet.de/event_sh-tourismus/e-day-spa-mit-meerblick-2?eventDateId=1980956&widgetToken=zOpiJSoCowQ.&
----------------------------------------
Category: sonstige Seminare & Kurse
Title: TIERISCH NORDISCH
Date and Location: 26.11.2024 / 08:00 - 18:00 Uhr / Galerie Premium Mobile Kuntz, Gettorf
Source: https://tashwhl.infomaxnet.de/event_sh-tourismus/e-tierisch-nordisch?eventDateId=2072317&widgetToken=zOpiJSoCowQ.&
----------------------------------------
Category: Ausstellungen
Title: WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT
Date and Location: 26.11.2024 / 08:00 - 17:00 Uhr / Kanzlei am Hafen, Travemünde
Source: https://tashwhl.infomaxnet.de/event_sh-tourismus/e-weit-mehr-als-meer-kunstwerke-aus-leidenschaft-3?eventDateId=1979988&widgetToken=zOpiJSoCowQ.&
----------------------------------------
Category: sons

NoSuchElementException: Message: Unable to locate element: p.-IMXEVNT-listElement__text__subline; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [7]:
# Find all article elements representing events
articles = driver.find_elements(By.CSS_SELECTOR, 'article.-IMXEVNT-listElement')

In [9]:
# List to store event data
events_data = []

# Iterate through each article and extract information
for article in articles:
    try:
        # Extract the required information
        event_category = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__subline').text
    except NoSuchElementException:
        event_category = 'N/A'

    try:
        event_title = article.find_element(By.CSS_SELECTOR, 'span.-IMXEVNT-title').get_attribute('data-uppertitle')
    except NoSuchElementException:
        event_title = 'N/A'

    try:
        event_date_location = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__info').text
    except NoSuchElementException:
        event_date_location = 'N/A'

    # Split the event_date_location into date, time, and location
    if event_date_location != 'N/A':
        parts = [x.strip() for x in event_date_location.split('/')]
    else:
        parts = []

    if len(parts) >= 3:
        date, time, location = parts[0], parts[1], ' / '.join(parts[2:])
    elif len(parts) == 2:
        date, time, location = parts[0], parts[1], ''
    else:
        date, time, location = 'N/A', 'N/A', 'N/A'

    # Split the location into location and city at the last comma
    if ',' in location:
        location, city = [x.strip() for x in location.rsplit(',', 1)]
    else:
        location, city = location, 'N/A'

    # Append the event data to the list
    events_data.append({
        'Category': event_category,
        'Title': event_title,
        'Date': date,
        'Time': time,
        'Location': location,
        'City': city
    })

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(events_data)

df

,Category,Title,Date,Time,Location,City
0,Sonstiges,DAY SPA MIT MEERBLICK,26.11.2024,07:00 - 22:00 Uhr,ATLANTIC Grand Hotel Travemünde,Travemünde
1,sonstige Seminare & Kurse,TIERISCH NORDISCH,26.11.2024,08:00 - 18:00 Uhr,Galerie Premium Mobile Kuntz,Gettorf
2,Ausstellungen,WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT,26.11.2024,08:00 - 17:00 Uhr,Kanzlei am Hafen,Travemünde
3,sonstige Seminare & Kurse,WINDART SKULPTUREN WALK,26.11.2024,08:00 Uhr,verschiedene Spielstätten in Travemünde,Travemünde
4,Ausstellungen,WINDART SKULPTUREN WALK,26.11.2024,08:00 - 23:00 Uhr,verschiedene Orte in Travemünde,Travemünde
...,...,...,...,...,...,...
779,Kulinarisch,WOCHENMARKT NEUSTADT IN HOLSTEIN - KAUF FRISCH...,20.12.2024,08:00 - 13:00 Uhr,Marktplatz Neustadt in Holstein,Neustadt in Holstein
780,sonstige Kunst & Kultur,"AUSSTELLUNG ""BEGEGNUNG"" DER KüNSTLERINNEN MEGI...",20.12.2024,09:00 - 12:00 Uhr,Rathaus,Heiligenhafen
781,sonstige Feste & Märkte,FISCH-FABRIKVERKAUF BEI CHRISTIAN GOEDEKEN JR....,20.12.2024,09:00 - 18:00 Uhr,Christian Goedeken jr. GmbH,Wentorf bei Hamburg
782,Ausstellungen,GäRTNERN IN DER STADT,20.12.2024,09:00 - 17:00 Uhr,Museum für Natur und Umwelt,Lübeck


In [41]:
df.to_csv("Scraped_Events_Schleswig_Holstein.csv")

In [12]:
driver.close()

### Function for Scraping

In [1]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

import pandas as pd

#import locale
#from datetime import datetime, timedelta
#import time

In [13]:
def scrape_sh_tourismus(days_in_advance=10):
    
    driver = webdriver.Firefox()
    driver.get('https://www.sh-tourismus.de/veranstaltungskalender')

    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe[src*="tashwhl.infomaxnet.de"]'))
    )
    driver.switch_to.frame(iframe)

    wait = WebDriverWait(driver, 10)
    clicks = days_in_advance*30 + 100 

    for i in range(clicks): 
        try:
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, '-IMXEVENT-lazyLoadButton')))
            button.click()
        except StaleElementReferenceException:
            continue
        except TimeoutException:
            print("Button not found or not clickable within the timeout period.")
            break

    articles = driver.find_elements(By.CSS_SELECTOR, 'article.-IMXEVNT-listElement')


    events_data = []

    for article in articles:
        try:
            event_category = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__subline').text
        except NoSuchElementException:
            event_category = 'N/A'

        try:
            event_title = article.find_element(By.CSS_SELECTOR, 'span.-IMXEVNT-title').get_attribute('data-uppertitle')
        except NoSuchElementException:
            event_title = 'N/A'

        try:
            event_date_location = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__info').text
        except NoSuchElementException:
            event_date_location = 'N/A'

        if event_date_location != 'N/A':
            parts = [x.strip() for x in event_date_location.split('/')]
        else:
            parts = []

        if len(parts) >= 3:
            date, time, location = parts[0], parts[1], ' / '.join(parts[2:])
        elif len(parts) == 2:
            date, time, location = parts[0], parts[1], ''
        else:
            date, time, location = 'N/A', 'N/A', 'N/A'

        if ',' in location:
            location, city = [x.strip() for x in location.rsplit(',', 1)]
        else:
            location, city = location, 'N/A'

        try:
            event_source = article.find_element(By.CSS_SELECTOR, 'h2 a').get_attribute('href')
        except NoSuchElementException:
            event_title = 'N/A'

        events_data.append({
            'Category': event_category,
            'Title': event_title,
            'Date': date,
            'Time': time,
            'Location': location,
            'City': city,
            'Source': event_source
        })

    df = pd.DataFrame(events_data)

    driver.close()

    return df


In [38]:
# Helper functions

def preprocess_time(time_str):
    if '-' in time_str:
        start_time, end_time = time_str.split(' - ')
        #start_time = start_time.replace(' Uhr', '')
        end_time = end_time.replace(' Uhr', '')
    else:
        start_time = time_str.replace(' Uhr', '')
        end_time = 'N/A'
    return pd.Series([start_time, end_time])


music = ['Konzert', 'konzert', 'Musik', 'musik', 'Party', 'party', 'Tanz', 'tanz', 'Festival', 'festival', 'Musical', 'musical', 'Jazz', 'jazz', 'Blues', 'blues', 'Country', 'country', 'Folk', 'folk', 'Rock', 'rock', 'Pop', 'pop', 'Klassik', 'klassik', 'Gospel', 'gospel', 'Chöre', 'chöre']

def check_music(category):
    for word in music:
        if word in category:
            return 'music'
    return 'no music'


In [43]:
def preprocess_sh_tourismus(df_raw):

    df_raw.rename(columns={'Title': 'Subject'}, inplace=True)

    df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)
    df_raw["End_date"] = df_raw["Start_date"]

    df_raw[['Start_time', 'End_time']] = df_raw['Time'].apply(preprocess_time)
    df_raw.drop(columns=['Time'], inplace=True)

    df_raw.rename(columns={'Source': 'Description'}, inplace=True)    

    df_raw['Music_label'] = df_raw['Category'].apply(check_music)

    df_prep = df_raw[['Subject','Start_date', 'End_date', 'Start_time', 'End_time', 'Location', 'City', 'Description', 'Category', 'Music_label']]
    return df_prep

In [42]:
df_raw = scrape_sh_tourismus(0)

In [44]:
df_raw

,Category,Title,Date,Time,Location,City,Source
0,Sonstiges,DAY SPA MIT MEERBLICK,26.11.2024,07:00 - 22:00 Uhr,ATLANTIC Grand Hotel Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...
1,sonstige Seminare & Kurse,TIERISCH NORDISCH,26.11.2024,08:00 - 18:00 Uhr,Galerie Premium Mobile Kuntz,Gettorf,https://tashwhl.infomaxnet.de/event_sh-tourism...
2,Ausstellungen,WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT,26.11.2024,08:00 - 17:00 Uhr,Kanzlei am Hafen,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...
3,sonstige Seminare & Kurse,WINDART SKULPTUREN WALK,26.11.2024,08:00 Uhr,verschiedene Spielstätten in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...
4,Ausstellungen,WINDART SKULPTUREN WALK,26.11.2024,08:00 - 23:00 Uhr,verschiedene Orte in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...
...,...,...,...,...,...,...,...
1003,Kulinarisch,WOCHENMARKT NEUSTADT IN HOLSTEIN - KAUF FRISCH...,20.12.2024,08:00 - 13:00 Uhr,Marktplatz Neustadt in Holstein,Neustadt in Holstein,https://tashwhl.infomaxnet.de/event_sh-tourism...
1004,sonstige Kunst & Kultur,"AUSSTELLUNG ""BEGEGNUNG"" DER KüNSTLERINNEN MEGI...",20.12.2024,09:00 - 12:00 Uhr,Rathaus,Heiligenhafen,https://tashwhl.infomaxnet.de/event_sh-tourism...
1005,sonstige Feste & Märkte,FISCH-FABRIKVERKAUF BEI CHRISTIAN GOEDEKEN JR....,20.12.2024,09:00 - 18:00 Uhr,Christian Goedeken jr. GmbH,Wentorf bei Hamburg,https://tashwhl.infomaxnet.de/event_sh-tourism...
1006,Ausstellungen,GäRTNERN IN DER STADT,20.12.2024,09:00 - 17:00 Uhr,Museum für Natur und Umwelt,Lübeck,https://tashwhl.infomaxnet.de/event_sh-tourism...


In [45]:
df_prep = preprocess_sh_tourismus(df_raw)
df_prep

,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,DAY SPA MIT MEERBLICK,26.11.2024,26.11.2024,07:00,22:00,ATLANTIC Grand Hotel Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Sonstiges,no music
1,TIERISCH NORDISCH,26.11.2024,26.11.2024,08:00,18:00,Galerie Premium Mobile Kuntz,Gettorf,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Seminare & Kurse,no music
2,WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT,26.11.2024,26.11.2024,08:00,17:00,Kanzlei am Hafen,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music
3,WINDART SKULPTUREN WALK,26.11.2024,26.11.2024,08:00,N/A,verschiedene Spielstätten in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Seminare & Kurse,no music
4,WINDART SKULPTUREN WALK,26.11.2024,26.11.2024,08:00,23:00,verschiedene Orte in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music
...,...,...,...,...,...,...,...,...,...,...
1003,WOCHENMARKT NEUSTADT IN HOLSTEIN - KAUF FRISCH...,20.12.2024,20.12.2024,08:00,13:00,Marktplatz Neustadt in Holstein,Neustadt in Holstein,https://tashwhl.infomaxnet.de/event_sh-tourism...,Kulinarisch,no music
1004,"AUSSTELLUNG ""BEGEGNUNG"" DER KüNSTLERINNEN MEGI...",20.12.2024,20.12.2024,09:00,12:00,Rathaus,Heiligenhafen,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Kunst & Kultur,no music
1005,FISCH-FABRIKVERKAUF BEI CHRISTIAN GOEDEKEN JR....,20.12.2024,20.12.2024,09:00,18:00,Christian Goedeken jr. GmbH,Wentorf bei Hamburg,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Feste & Märkte,no music
1006,GäRTNERN IN DER STADT,20.12.2024,20.12.2024,09:00,17:00,Museum für Natur und Umwelt,Lübeck,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music


In [28]:
df_raw

,Category,Subject,Start_date,Location,City,Description,End_date,Start_time,End_time
0,Sonstiges,DAY SPA MIT MEERBLICK,26.11.2024,ATLANTIC Grand Hotel Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,26.11.2024,07:00,22:00
1,sonstige Seminare & Kurse,TIERISCH NORDISCH,26.11.2024,Galerie Premium Mobile Kuntz,Gettorf,https://tashwhl.infomaxnet.de/event_sh-tourism...,26.11.2024,08:00,18:00
2,Ausstellungen,WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT,26.11.2024,Kanzlei am Hafen,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,26.11.2024,08:00,17:00
3,sonstige Seminare & Kurse,WINDART SKULPTUREN WALK,26.11.2024,verschiedene Spielstätten in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,26.11.2024,08:00,N/A
4,Ausstellungen,WINDART SKULPTUREN WALK,26.11.2024,verschiedene Orte in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,26.11.2024,08:00,23:00
...,...,...,...,...,...,...,...,...,...
951,Kulinarisch,WOCHENMARKT NEUSTADT IN HOLSTEIN - KAUF FRISCH...,20.12.2024,Marktplatz Neustadt in Holstein,Neustadt in Holstein,https://tashwhl.infomaxnet.de/event_sh-tourism...,20.12.2024,08:00,13:00
952,sonstige Kunst & Kultur,"AUSSTELLUNG ""BEGEGNUNG"" DER KüNSTLERINNEN MEGI...",20.12.2024,Rathaus,Heiligenhafen,https://tashwhl.infomaxnet.de/event_sh-tourism...,20.12.2024,09:00,12:00
953,sonstige Feste & Märkte,FISCH-FABRIKVERKAUF BEI CHRISTIAN GOEDEKEN JR....,20.12.2024,Christian Goedeken jr. GmbH,Wentorf bei Hamburg,https://tashwhl.infomaxnet.de/event_sh-tourism...,20.12.2024,09:00,18:00
954,Ausstellungen,GäRTNERN IN DER STADT,20.12.2024,Museum für Natur und Umwelt,Lübeck,https://tashwhl.infomaxnet.de/event_sh-tourism...,20.12.2024,09:00,17:00


In [16]:
df_raw.rename(columns={'Title': 'Subject'}, inplace=True)

In [ ]:
df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)

In [ ]:
df_raw["End_date"] = df_raw["Start_date"]

In [22]:
def preprocess_time(time_str):
    if '-' in time_str:
        start_time, end_time = time_str.split(' - ')
        #start_time = start_time.replace(' Uhr', '')
        end_time = end_time.replace(' Uhr', '')
    else:
        start_time = time_str.replace(' Uhr', '')
        end_time = 'N/A'
    return pd.Series([start_time, end_time])

In [23]:
df_raw[['Start_time', 'End_time']] = df_raw['Time'].apply(preprocess_time)

In [25]:
df_raw.drop(columns=['Time'], inplace=True)

In [ ]:
df_raw.rename(columns={'Source': 'Description'}, inplace=True)

In [32]:
df_raw['Category'].unique()

array(['Sonstiges', 'sonstige Seminare & Kurse', 'Ausstellungen',
       'Kulinarisch', 'N/A', 'sonstige Kunst & Kultur',
       'Weihnachtsmärkte', 'Gottesdienste', 'Spielen, Quizzen & Gaming',
       'Best-Of, Feste, Highlights, Kulinarisch, Stadtfeste & Märkte, Weihnachtsmärkte, Winter',
       'sonstige Ausflüge & Führungen', 'sonstige Religion',
       'Radsport, Rundfahrten',
       'Abend / Nacht, Bernsteinschleifen / Basteln, Familie / Kinder, Führung, Indoor',
       'Familie / Kinder, Führung, Outdoor', 'Natur',
       'Führung, Outdoor, Wattwanderung',
       'sonstige Feste & Märkte, Weihnachtsmärkte', 'Segeln & Yachting',
       'sonstige Feste & Märkte',
       'Familie / Kinder, Führung, Outdoor, Wattwanderung',
       'Familie & Kinder',
       'Barrierefrei, Bernsteinschleifen, Bernsteinschleifen / Basteln, Familie & Kinder, Familie / Kinder, Indoor, Malen & Basteln',
       'Familie & Kinder, Familie / Kinder, Führung, Indoor, Nationalpark & Wattenmeer, Naturkunde, Ou

In [33]:
music = ['Konzert', 'konzert', 'Musik', 'musik', 'Party', 'party', 'Tanz', 'tanz', 'Festival', 'festival', 'Musical', 'musical', 'Jazz', 'jazz', 'Blues', 'blues', 'Country', 'country', 'Folk', 'folk', 'Rock', 'rock', 'Pop', 'pop', 'Klassik', 'klassik', 'Gospel', 'gospel', 'Chöre', 'chöre']

In [34]:
def check_music(category):
    for word in music:
        if word in category:
            return 'music'
    return 'no music'

In [35]:
df_raw['Music_label'] = df_raw['Category'].apply(check_music)

In [37]:
df_raw = df_raw[['Subject','Start_date', 'End_date', 'Start_time', 'End_time', 'Location', 'City', 'Description', 'Category', 'Music_label']]
df_raw

,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,DAY SPA MIT MEERBLICK,26.11.2024,26.11.2024,07:00,22:00,ATLANTIC Grand Hotel Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Sonstiges,no music
1,TIERISCH NORDISCH,26.11.2024,26.11.2024,08:00,18:00,Galerie Premium Mobile Kuntz,Gettorf,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Seminare & Kurse,no music
2,WEIT MEHR ALS MEER | KUNSTWERKE AUS LEIDENSCHAFT,26.11.2024,26.11.2024,08:00,17:00,Kanzlei am Hafen,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music
3,WINDART SKULPTUREN WALK,26.11.2024,26.11.2024,08:00,N/A,verschiedene Spielstätten in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Seminare & Kurse,no music
4,WINDART SKULPTUREN WALK,26.11.2024,26.11.2024,08:00,23:00,verschiedene Orte in Travemünde,Travemünde,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music
...,...,...,...,...,...,...,...,...,...,...
951,WOCHENMARKT NEUSTADT IN HOLSTEIN - KAUF FRISCH...,20.12.2024,20.12.2024,08:00,13:00,Marktplatz Neustadt in Holstein,Neustadt in Holstein,https://tashwhl.infomaxnet.de/event_sh-tourism...,Kulinarisch,no music
952,"AUSSTELLUNG ""BEGEGNUNG"" DER KüNSTLERINNEN MEGI...",20.12.2024,20.12.2024,09:00,12:00,Rathaus,Heiligenhafen,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Kunst & Kultur,no music
953,FISCH-FABRIKVERKAUF BEI CHRISTIAN GOEDEKEN JR....,20.12.2024,20.12.2024,09:00,18:00,Christian Goedeken jr. GmbH,Wentorf bei Hamburg,https://tashwhl.infomaxnet.de/event_sh-tourism...,sonstige Feste & Märkte,no music
954,GäRTNERN IN DER STADT,20.12.2024,20.12.2024,09:00,17:00,Museum für Natur und Umwelt,Lübeck,https://tashwhl.infomaxnet.de/event_sh-tourism...,Ausstellungen,no music


In [ ]:
df_raw.to_csv("Scraped_Events_Schleswig_Holstein.csv")